# Tutorial 1: 2D pressure fluctuations in a turbulent jet

In this tutorial we explore a small dataset provided with this package that contains the flow exiting a nozzle (also referred to as a jet). The data is two-dimensional and it is provided in equally-spaced cylindrical coordinates _(r,x)_.

## Import required libraries

The first step is to import the required libraries.

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import os
import sys
import numpy as np

# Current, parent and file paths
CWD = os.getcwd()
CFD = os.path.abspath('')

# project libraries
sys.path.append(os.path.join(CFD,"../../"))

# Import library specific modules
from pyspod.spod.standard  import Standard  as spod_standard
from pyspod.spod.streaming import Streaming as spod_streaming
import pyspod.spod.utils     as utils_spod
import pyspod.utils.weights  as utils_weights
import pyspod.utils.errors   as utils_errors
import pyspod.utils.io       as utils_io
import pyspod.utils.postproc as post

## Setup MPI Communicator for parallel computation (if available)

The second steps consists of setting up the MPI Communicator, if we want to run a parallel computation. 
This step requires that the user has a version of MPI installed in their machine / HPC system. Currently tested MPI platforms include Open MPI, and mpich. 

In [3]:
## -------------------------------------------------------------------
## initialize MPI
## -------------------------------------------------------------------
try:
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    rank = comm.rank
except:
    comm = None
    rank = 0
## -------------------------------------------------------------------

## Read data and parameters

We can now read the data and the parameters from `fluidmechanics_data.mat`, and `input_tutorial1.yaml`, respectively. Both files are in `tests/data`. The input file contains some parameters that are required and some that are optional. 

In detail:

```yaml
required:
  - time_step   : 1
  - n_space_dims: 2
  - n_variables : 1
  - n_dft       : 64

optional:
  - overlap          : 50
  - mean_type        : 'longtime'
  - normalize_weights: False
  - normalize_data   : False
  - n_modes_save     : 40
  - conf_level       : 0.95
  - reuse_blocks     : False
  - savefft          : False
  - dtype            : 'double'
  - savedir          : 'results'
  - fullspectrum     : False

```

The **required parameters** are:
- `time_step`, the time step of the data, 
- `n_space_dims`, the number of spatial dimensions of the data; for 2D data this number is equal to 2, 
- `n_variables`, the number of variables; for one single field, e.g., pressure, this number is equal to 1,
- `n_dft`, the number of time snapshots in each block for the data partitioning; for this tutorial, we are using 64 time snapshots, for a dataset containing 1000 time snapshots. This means that our data will be divided into blocks of 64 time snapshots each.

The **optional parameters** are:
- `overlap`, that is whether to use an overlap region between blocks (in percentage),
- `mean_type`, the type of mean to be used; this can either be the mean of all time snapshots (`longtime`), or the the mean of each block (`blockwise`); it can also be set to `zero`, if the mean was already removed from the data,
- `normalize_weights`, whether or not to normalize the weights,
- `normalize_data`, whether or not to normalize the data,
- `n_modes_save`, the number of modes to be saved,
- `conf_level`, the confidence interval for the eigenvalues,
- `reuse_blocks`, whether to reuse pre-computed FFT blocks (if present),
- `savefft`, whether to save the FFT blocks,
- `dtype`, the data type to be used (either `single` or `double` precision),
- `savedir`, where to save the results,
- `fullspectrum`, whether to use a full spectrum (instead of half) for the FFT.

In [12]:
## -------------------------------------------------------------------
## read data and params
## -------------------------------------------------------------------
## data
data_file = os.path.join(CFD, '../../tests/data', 'fluidmechanics_data.mat')
data_dict = utils_io.read_data(data_file=data_file)
data = data_dict['p'].T
dt = data_dict['dt'][0,0]
nt = data.shape[0]
x1 = data_dict['r'].T; x1 = x1[:,0]
x2 = data_dict['x'].T; x2 = x2[0,:]
## params
config_file = os.path.join(CFD, '../../tests/data', 'input_tutorial1.yaml')
params = utils_io.read_config(config_file)
params['time_step'] = dt
## -------------------------------------------------------------------

## Compute SPOD modes

We can then initialize the SPOD algorithms, both the batch (`standard`) and the streaming (`streaming`) one:

- `standard  = spod_standard (params=params, comm=comm)`
- `streaming = spod_streaming(params=params, comm=comm)`

and compute the **SPOD modes $\phi(\mathbf{x})$**, using `spod = standard.fit(data_list=data)`.

We can also check the orthogonality of the modes using `check_orthogonality`.
The modes provided should return `flag = True`, and `ortho < 1e-15`.

In [15]:
## -------------------------------------------------------------------
## compute spod modes and check orthogonality
## -------------------------------------------------------------------
standard  = spod_standard(params=params, comm=comm)
streaming = spod_streaming(params=params, comm=comm)
spod = standard.fit(data_list=data)
results_dir = spod.savedir_sim
flag, ortho = utils_spod.check_orthogonality(
    results_dir=results_dir, mode_idx1=[1],
    mode_idx2=[0], freq_idx=[5], dtype='double',
    comm=comm)
print(f'flag = {flag},  ortho = {ortho}')
## -------------------------------------------------------------------

 
Initialize data ...
 
Initialize data
------------------------------------
- reading first time snapshot for data dimensions
- getting data dimensions
nx: 1760
dim: 4
shape: (1, 20, 88, 1)
xdim: 2
xshape: (20, 88)
nt: 1000
- checking weight dimensions
- distributing data (if parallel)
--- reading data (1 stage reader) finished in 0.01 s
- loaded data into memory: 1.1920928955078125e-06 s.
- computed mean: 0.0010142326354980469 s.
SPOD parameters
------------------------------------
Problem size (real)      : 0.01 GB (min 0.01 GB/proc, max 0.01 GB/proc)
Problem size (complex)   : 0.03 GB (min 0.03 GB/proc, max 0.03 GB/proc)
Data type for real       : <class 'numpy.float64'>
Data type for complex    : <class 'numpy.complex128'>
No. snapshots per block  : 64
Block overlap            : 32
No. of blocks            : 30
Windowing fct. (time)    : hamming
Weighting fct. (space)   : uniform
Mean                     : longtime
Number of frequencies    : 33
Time-step                : 0.2
Time 

## Compute time coefficients

Once the modes are computed, we can calculate the **time coefficients $a(t)$**

In [ ]:
## -------------------------------------------------------------------
## compute coefficients
## -------------------------------------------------------------------
file_coeffs, coeffs_dir = utils_spod.compute_coeffs_op(
    data=data, results_dir=results_dir, comm=comm)
## -------------------------------------------------------------------

## Reconstruct original data

From the SPOD modes $\phi(\mathbf{x})$ and the time coefficients $a(t)$, we can finally **reconstruct the original data**

In [ ]:
## -------------------------------------------------------------------
## compute reconstruction
## -------------------------------------------------------------------
file_dynamics, coeffs_dir = utils_spod.compute_reconstruction(
    coeffs_dir=coeffs_dir, time_idx='all', comm=comm)
## -------------------------------------------------------------------

Finally, we can perform some **postprocessing** and **visualization**

## Postprocessing and visualization

### Eigenvalues

We can first plot the eigenvalues in the complex plane, as well as the eigenvalues vs frequency and period.

In [ ]:
## only rank 0
if rank == 0:
    ## plot eigenvalues
    spod.plot_eigs()
    spod.plot_eigs_vs_frequency()
    spod.plot_eigs_vs_period()


### SPOD modes

We can then plot the SPOD modes at some prescribed frequencies (equivalently periods) of interest. 
In this case we chose 
- period 1, T1 = 0.9, and 
- period 2, T2 = 4.

In [ ]:
## only rank 0
if rank == 0:
    ## identify frequency of interest
    T1 = 0.9; T2 = 4
    f1, f1_idx = spod.find_nearest_freq(freq_req=1/T1, freq=spod.freq)
    f2, f2_idx = spod.find_nearest_freq(freq_req=1/T2, freq=spod.freq)

    ## plot 2d modes at frequency of interest
    spod.plot_2d_modes_at_frequency(freq_req=f1, freq=spod.freq,
        modes_idx=[0,1,2], x1=x2, x2=x1,
        equal_axes=True)

    ## plot 2d modes at frequency of interest
    spod.plot_2d_modes_at_frequency(freq_req=f2, freq=spod.freq,
        modes_idx=[0,1,2], x1=x2, x2=x1,
        equal_axes=True)

### Time coefficients

We can also visualize the time coefficients

In [ ]:
## only rank 0
if rank == 0:    
    ## plot coefficients
    coeffs = np.load(file_coeffs)
    post.plot_coeffs(coeffs, coeffs_idx=[0,1],
        path=results_dir)

### Reconstruction of the solution

We then visualize the reconstruction of the data based on the SPOD modes and time coefficients.

In [ ]:
## only rank 0
if rank == 0:
    ## plot reconstruction
    recons = np.load(file_dynamics)
    post.plot_2d_data(recons, time_idx=[0,10],
        path=results_dir, x1=x2, x2=x1, equal_axes=True)



### The original data

We can also visualize the original data, both in terms of 2D contours or data tracers.

In [ ]:
## only rank 0
if rank == 0:
    ## plot data
    data = spod.get_data(data)
    post.plot_2d_data(data, time_idx=[0,10],
        path=results_dir, x1=x2, x2=x1, equal_axes=True)
    post.plot_data_tracers(data, coords_list=[(5,0.5)],
        time_limits=[0,nt], path=results_dir)
    ## -------------------------------------------------------------